# Regression-House-Pricing 

Let's study regression by using Light Gradient Boosting and different variations of a dataset:

## Simple case

### Only Null Removal 

### Null Removal + Imputation 

### Null Removal + Impatition + Encoding 

## K-Fold Validation 

## Parameter optimization